<a id='top'></a>

In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')


# %matplotlib nbagg 
%matplotlib notebook
# %matplotlib inline



## Third party 
import numpy as np
import os, time, zarr, sys
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import matplotlib as mpl

import unslice.IO as io
from unslice.utils import *
from unslice.registration.featmatch import *
from unslice.registration.transform import *
from unslice.registration.rigid import *
from unslice.registration.gpu_transform import *
from unslice.registration.utils import *
from unslice.segmentation import *
from unslice.tracing.pyoof import OOF, apply_oof_v2
from unslice.tracing.skel import *
from unslice.flatten import *
from unslice.lightsheetcorrect import *




In [2]:
# Parameters that are constant throughout notebook
working_dir = '/mnt/share3/webster/mEhmAD_1-3_real/tracing'

def bdir(fname):
    return os.path.join(working_dir, fname)

# prefix to add to the beginning of each filename 
name_prefix = '1-ptau_pred'

# Table of contents

### Pre-processing
[1. Convert to zarr](#convert)<br>
[2. Flatten warp](#flattenwarp)<br>
[3. Lectin warp](#anchorwarp)<br>

# Convert to zarr
<a id='convert'></a>

[Return to top](#top)

In [3]:
# 1

slab_path = bdir('1-ptau_pred.tif')
slab_zarr_path = bdir(name_prefix+'.zarr')
pc2_img_size = (1024,)*3


## Optional parameters 
load_num_slices = 40 # should be smaller than z chunk size, default None 
resample_num_slices = 1 # number of slices to be processed in one go for resampling, default 1
resample_factor = (1,1) # original is (0.306,0.433,0.306)
chunks = (200,200,200) 
num_workers = 20 
file_names = None #'img_[0-9]{4}.tiff' # default 'img_[0-9]{4}.tiff'

## crop
xrange = None
yrange = None
zrange = None

## rotate
lateral_rotate_angle = None # default None

## flip
flip = (0,0,0) # default (0,0,0)


#############
start = time.time()
utils.convert_to_zarr_v2(slab_path, slab_zarr_path, pc2_img_size, load_num_slices=load_num_slices,
                        resample_num_slices=resample_num_slices, file_names=file_names, 
                        chunks=chunks, num_workers=num_workers, lateral_rotate_angle=lateral_rotate_angle,
                        flip=flip, crop_xcoords=xrange, crop_ycoords=yrange, crop_zcoords=zrange,
                        resample_factor=resample_factor)
print(time.time()-start, 'seconds elapsed')

Reading image
66.91312980651855 seconds elapsed


## Flatten warp the slabs

[Return to top](#top)

<a id='flattenwarp'></a>

In [4]:
# (1)
grid_path = '/mnt/share3/webster/mEhmAD_1-3_real/warping_grids/1-lec_grid_flatten_upsampled.npy'
original_shape = (18100,8100,5336)
new_grid_spacing = 3*(64,) # doesn't matter if range_mode= 'moving'
range_mode='moving'

# We will compute  the grid spacing from the zarr specifically 
save_grid_path = '/mnt/share3/webster/mEhmAD_1-3_real/warping_grids/1-lec_grid_flatten_upsampled_cropped.npy'

# these are the coordinates in original image of FOV 
xrange = [6382,7406]
yrange = [5915,6939]
zrange = [2976,4000]



################
grid_new,new_shape = crop_grid(grid_path, original_shape, new_grid_spacing, range_mode=range_mode, 
                         xrange=xrange, yrange=yrange, zrange=zrange, save_grid_path=save_grid_path)

New fixed image shape: (1095, 1167, 1367)
New fixed range: x 6312 7407
New fixed range: y 5896 7063
New fixed range: z 3318 4685


In [6]:
# How much to translate when overlaying with cropped flattened FOV
xrange = [5759,8410]
yrange = [5981,7328]
zrange = [4213-1500,4213+500]
print(6312-5759)
print(5896-5981)
print(3318-(4213-1500))

553
-85
605


In [5]:
# 3
fixed_zarr_path = new_shape
moving_zarr_path = bdir(name_prefix+'.zarr')
warped_zarr_path = bdir(name_prefix+'_flattened.zarr')


# Parameters for TPS zarr warp
grid_spacing = 3*(64,)
chunks=3*(200,)
nb_workers = 8


# grid I/O 
save_grid_values_path = None
use_grid_values_path = '/mnt/share3/webster/mEhmAD_1-3_real/warping_grids/1-lec_grid_flatten_upsampled_cropped.npy'




##########################

moving_pts_paths = None # doesn't actually matter for this 
fixed_pts_paths =  None

# anchor parameters (using the surface on the other side and manually identified anchors on the cut surface)
static_pts_paths = None
# affine parameters 
R_path = None
b_path = None
zadd = new_shape[2]-zarr.open(moving_zarr_path).shape[2]

TPS_warp(moving_zarr_path, fixed_zarr_path, warped_zarr_path, moving_pts_paths, fixed_pts_paths,
         static_pts_paths=static_pts_paths, R_path=R_path, b_path=b_path,
          grid_spacing=grid_spacing, smooth=2, chunks=chunks,zadd=zadd,
          nb_workers=nb_workers, padding=2, save_grid_values_path=save_grid_values_path, 
          show_residuals=True, use_grid_values_path=use_grid_values_path)

# Convert zarr to tiff
tiff_path = warped_zarr_path[:-5]+'_tiffs'
convert_zarr_to_tiff(warped_zarr_path, tiff_path, num_workers=24)

(1095, 1167, 1367)
Loading grid values...
Warping image...
Moving image size: 2.147483648 GB


100%|██████████| 252/252 [00:33<00:00, 18.47it/s]


Time elapsed: 0.904174 minutes
Loading z 0 - 200


100%|██████████| 200/200 [00:05<00:00, 36.43it/s]

Loading z 200 - 400


...


100%|██████████| 200/200 [00:05<00:00, 36.71it/s]


Loading z 1000 - 1200


100%|██████████| 200/200 [00:08<00:00, 24.77it/s]


Loading z 1200 - 1367


100%|██████████| 167/167 [00:04<00:00, 37.22it/s]


# Get surface endpoints

In [7]:
# Now set the parameters 
# Need one set for the top surface and one set for the bottom surface of each slab 

zarr_path = bdir(name_prefix+'_flattened.zarr')
zarr_segmented_path = bdir(name_prefix+'_flattened_segmented.zarr') # bdir('vessel_tests/'+name_prefix+'_segmented.zarr')

sample_coord_ranges = None #[[[0,1000],[0,1000],[0,400]]]
threshold_type = "set" # "set" or "graphcut" or "otsu"

if threshold_type == 'set':
    intensity_thresholds = [60000]
    morphopts = [None]

    for i in range(len(intensity_thresholds)):
        opts = {
            'percentile_threshold': intensity_thresholds[i],
            'threshold_type': threshold_type,
            'morphopts': morphopts[i],
            'num_workers': 24,
            'sample_coord_ranges': sample_coord_ranges
        }
        
        threshold_image(zarr_path, zarr_segmented_path, **opts)
        
elif threshold_type == 'graphcut':
    min_thresholds = [.25*65535]
    saturate_image_thresholds = [.95*65535]
    morphopts = [None]  #[('dilate','ball',2)]*2
    
    for i in range(len(min_thresholds)):
        opts = {
            'min_threshold' : min_thresholds[i], # This sets a minimum intensity threshold for pixels 
            'saturate_image_threshold' : saturate_image_thresholds[i],
            'k' : 2,
            'alpha' : 0.25,
            'num_workers' : 24,
            'overlap' : 25,
            'morphopts' : morphopts[i],
            'sample_coord_ranges' : sample_coord_ranges 
            }
        zarr_graphcut3d(zarr_path, zarr_segmented_path, **opts)

100%|██████████| 252/252 [00:22<00:00, 11.20it/s]


In [18]:
surf_zarr_path = None 
segmented_path = bdir(name_prefix+'_flattened_segmented.zarr')
skel_path = bdir(name_prefix+'_flattened_segmented_skel.zarr')
endpoints_path = bdir(name_prefix+'_endpoints_bottom.npy')
min_branch_length = 15
min_cc = 100
overlap = 25 # make sure this is much greater than the min_cc and min_branch_length 
num_workers = 24

# Kwargs for prune_directionality 
prune_directionality_num_points = 4
orientation = 'bottom' # first we get all the top surface points 


############# Perform skeletonization ##############
start = time.time()
top_endpoints = trace_zarr(segmented_path, surf_zarr_path, skel_path,
                      min_branch_length, min_cc, overlap=overlap, num_workers=num_workers,
                      prune_directionality_num_points=prune_directionality_num_points,
                      orientation=orientation) # , z_shape=z_shape)
np.save(endpoints_path,top_endpoints)
print("Time elapsed: %f hours" %((time.time()-start)/3600))



100%|██████████| 252/252 [00:31<00:00,  8.05it/s]


Time elapsed: 0.016544 hours


In [19]:
# view skeleton

z = zarr.open(skel_path)
img = z[:,85:,:]

io.writeData(skel_path[:-5]+'.tif',img)

'/mnt/share3/webster/mEhmAD_1-3_real/tracing/1-ptau_pred_flattened_segmented_skel.tif'

In [10]:
# view segmentation

z = zarr.open(segmented_path)
img = z[:]

io.writeData(segmented_path[:-5]+'.tif',img)

'/mnt/share3/webster/mEhmAD_1-3_real/tracing/1-ptau_pred_flattened_segmented.tif'